In [ ]:
from atlassian.bitbucket.cloud import Cloud
import os, json
import requests
import pandas as pd

username = os.getenv('BITBUCKET_USER', None)
password = os.getenv('BITBUCKET_PASS', None)
url = os.getenv('BITBUCKET_URL', None)
workspace = os.getenv('BITBUCKET_WORKSPACE', None)
project_key = os.getenv('PROJECT_KEY', None)
user_id = os.getenv('USER_ID', None)


In [ ]:

headers = {
    'Content-Type': 'application/json',
}
params = (
    ('key1', 'mykeyhere'),
)
url = f'https://api.bitbucket.org/2.0/repositories/{workspace}/?q=project.key="{project_key}"&limit=1000&pagelen=100'


def get_repo(repo_name):
    url = f'https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_name}'
    rsp = requests.get(url,  headers=headers, auth=(username, password))
    print(json.dumps(rsp.json(), sort_keys=True, indent=4)) 
    
def get_repo_default_reviewers(repo_name):
    url = f'https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_name}/default-reviewers'
    rsp = requests.get(url,  headers=headers, auth=(username, password))
    print(json.dumps(rsp.json(), sort_keys=True, indent=4)) 

def put_repo_default_reviewers(repo_name, target_username_uuid):
    url = f'https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_name}/default-reviewers/{target_username_uuid}'
    print(url)
    rsp = requests.put(url, data = { 'target_username':target_username_uuid}, auth=(username, password))
    print(f'{rsp.status_code} {rsp.text} {rsp.reason}')

def get_repo_branch_restrictions(repo_name):
    url = f'https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_name}/branch-restrictions'
    print(url)
    rsp = requests.get(url,
        headers = headers, 
        auth=(username, password))
    print(json.dumps(rsp.json(), sort_keys=True, indent=4)) 

## to be completed
def put_repo_branch_restrictions(repo_name):
    url = f'https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_name}/branch-restrictions'
    print(url)
    rsp = requests.put(url,
        data = { 'target_username':target_username_uuid}, 
        auth=(username, password))
    print(f'{rsp.status_code} {rsp.text} {rsp.reason}')

def _main():
    headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
    repos_request = requests.get(url,  headers=headers,auth=(username, password))
    repos_json = repos_request.json()
    df = pd.DataFrame.from_dict(repos_json['values'])

    for repo in df['name']:
        #get_repo_default_reviewers(repo)
        if repo == 'ta-common':
            put_repo_default_reviewers(repo, user_id)
        if repo == 'ta-dealing-service':
            get_repo_branch_restrictions(repo)
_main()

In [ ]:
bitbucket = Cloud(
    url=url,
    username = username,
    password = password,
    cloud=True)

index = 0
project = bitbucket.workspaces.get(workspace).projects.get('XLO')

for r in project.repositories.each():
    print(f"Repo {r.name}")
    for dr in r.default_reviewers.each():
            print("   Default reviewer " + dr.nickname)
    for br in r.branch_restrictions.each():
        print(br)
    
#for r in project.repos.

In [ ]:
import json
from types import SimpleNamespace

# Parse JSON into an object with attributes corresponding to dict keys.
x = json.loads(r.text, object_hook=lambda d: SimpleNamespace(**d))
print(x[0])